In [12]:
import pandas as pd

# Multi-scale-alignment

- Alignement of spatial resolution in modelsetup data
- Part of the energytransitionmodelling effort

@author: Mark Hupkens
@date: 16-05-2019

In [13]:
'''Import files'''
df_gem = pd.read_excel("D:/markhupkens/Downloads/ModelSetUpEnergyNL02EPMH2.xlsx", sheet_name='gemeente')
df_buurt = pd.read_excel("D:/markhupkens/Downloads/ModelSetUpEnergyNL02EPMH2.xlsx", sheet_name='buurt')
df_wijk = pd.read_excel("D:/markhupkens/Downloads/ModelSetUpEnergyNL02EPMH2.xlsx", sheet_name='wijk')
df_mod = pd.read_excel("D:/markhupkens/Downloads/ModelSetUpEnergyNL02EPMH2.xlsx", sheet_name='ModelSpecification')  

### Q: how do model scales line up in the data?

In [14]:
df_wijk.Entities.nunique()

2863

In [15]:
df_buurt.Mapping.nunique()

3021

A: There are more wijken included as a mapping on buurt-level, than as there are entities on wijk-level
A2: same applies for municipalities

## Allign data resolutions

In [16]:
'''Check and keep only rows if entities and mappings line up'''

# Keep only gemeenten that are mappings in wijk
df_gem_new = df_gem.loc[df_gem.Entities.isin(df_wijk.Mapping)==True]

# Keep only wijk rows whose mapping is an entity in gemeente
df_wijk_new = df_wijk.loc[df_wijk.Mapping.isin(df_gem_new.Entities)==True]

# Keep only wijk rows whose entity is a mapping in df_buurt
df_wijk_new = df_wijk_new.loc[df_wijk_new.Entities.isin(df_buurt.Mapping)==True]

# Keep only buurten whose mapping is an entity in df_wijk
df_buurt_new = df_buurt.loc[df_buurt.Mapping.isin(df_wijk_new.Entities)==True]

In [17]:
print('df_buurt_new has length',len(df_buurt_new))
print('df_wijk_new has length', len(df_wijk_new))
print('df_gem has length',len(df_gem_new))

df_buurt_new has length 10786
df_wijk_new has length 2532
df_gem has length 331


### Check new data resolutions

In [18]:
# buurt to wijk
(df_buurt_new.Mapping.isin(df_wijk_new.Entities)==True).value_counts()

True    10786
Name: Mapping, dtype: int64

In [19]:
# Wijk to gem
(df_wijk_new.Mapping.isin(df_gem_new.Entities)==True).value_counts()

True    2532
Name: Mapping, dtype: int64

In [20]:
# Gem to wijk
(df_gem_new.Entities.isin(df_wijk_new.Mapping)==True).value_counts()

True    331
Name: Entities, dtype: int64

All resolutions now line up, as all mappings in a low-resolution dataframe are included as entities in a high-resolution dataframe

### Data completeness

How much data has been lost in the alignment process

In [29]:
print(len(df_gem)-len(df_gem_new),'Gemeenten have been lost', (len(df_gem)-len(df_gem_new))/len(df_gem)*100 )
print(len(df_wijk)-len(df_wijk_new),'Wijken have been lost', (len(df_wijk)-len(df_wijk_new))/len(df_wijk)*100)
print(len(df_buurt)-len(df_buurt_new),'Buurten have been lost', (len(df_buurt)-len(df_buurt_new))/len(df_buurt)*100)

9 Gemeenten have been lost 2.6470588235294117
331 Wijken have been lost 11.561299336360461
1787 Buurten have been lost 14.212996102759881


### Export data to new stup file

In [22]:
# export as xlsx to genereate new modelspecification file
from pandas import ExcelWriter

with pd.ExcelWriter('D:/markhupkens/Dropbox/EnTransitionNL/0. Data/ModelSetUpEnergyNL02EPMH2Aligned.xlsx') as writer:  # doctest: +SKIP
    df_buurt_new.to_excel(writer, sheet_name='buurt')
    df_wijk_new.to_excel(writer, sheet_name='wijk')
    df_gem_new.to_excel(writer, sheet_name='gemeente')
    df_mod.to_excel(writer, sheet_name='ModelSpecification')

In [24]:
df_buurt_new

,Entities,Mapping,Bevolking buurt,Mannen buurt,Vrouwen buurt,Bevolking 0 tot 15 jaar buurt,Bevolking 15 tot 25 jaar buurt,Bevolking 25 tot 45 jaar buurt,Bevolking 45 tot 65 jaar buurt,Bevolking 65 jaar of ouder buurt,...,Gemiddeld gasgebruik hoekwoning m3 2017 buurt,Gemiddeld gasgebruik 2o1kap m3 2017 buurt,Gemiddeld gasgebruik vrijstaande woning m3 2017 buurt,Gemiddeld elektriciteitsgebruik appartement kWh 2017 buurt,Gemiddeld elektriciteitsgebruik hoekwoning kWh 2017 buurt,Gemiddeld elektriciteitsgebruik 2o1kap kWh 2017 buurt,Gemiddeld elektriciteitsgebruik tussenwoning kWh 2017 buurt,Gemiddeld elektriciteitsgebruik vrijstaande woning kWh 2017 buurt,Aandeel stadsverwarming pc 2017 buurt,Opgesteld vermogen geregistreerde PV panelen per adres locatie gem kWpiek 2018 buurt
0,Annen BU16800000,Wijk 00 Annen WK168000,3415,1660,1755,560,320,595,1100,830,...,1580.0000,1690.0000,2120.0000,1630.0000,2860.0000,3300.0000,2480.0000,3720.0000,0.0,7.0000
1,Verspreide huizen Annen BU16800009,Wijk 00 Annen WK168000,145,75,70,10,10,15,70,30,...,0.4242,0.4242,2340.0000,0.4242,0.4242,0.4242,0.4242,4830.0000,0.0,7.0000
2,Eext BU16800100,Wijk 01 Eext WK168001,1260,640,625,200,125,255,420,245,...,1290.0000,1700.0000,2270.0000,2060.0000,2440.0000,3150.0000,1970.0000,3590.0000,0.0,4.0000
3,Verspreide huizen Eext BU16800109,Wijk 01 Eext WK168001,170,90,85,5,10,20,75,45,...,0.4242,0.4242,2380.0000,0.4242,0.4242,0.4242,0.4242,3340.0000,0.0,6.0000
4,Anloo BU16800200,Wijk 02 Anloo WK168002,325,155,170,30,30,45,115,90,...,0.4242,1510.0000,2350.0000,0.4242,0.4242,2600.0000,0.4242,3760.0000,0.0,6.0000
5,Verspreide huizen Anloo BU16800209,Wijk 02 Anloo WK168002,125,75,55,10,10,15,50,25,...,0.4242,880.0000,2430.0000,0.4242,0.4242,2280.0000,0.4242,4690.0000,0.0,5.0000
6,Gasteren BU16800300,Wijk 03 Gasteren WK168003,390,200,190,25,55,40,150,105,...,2310.0000,1810.0000,2440.0000,0.4242,3540.0000,2860.0000,3190.0000,3740.0000,0.0,7.0000
7,Verspreide huizen Gasteren BU16800309,Wijk 03 Gasteren WK168003,15,5,10,0,0,0,0,0,...,0.4242,0.4242,0.4242,0.4242,0.4242,0.4242,0.4242,0.4242,0.0,0.4242
8,Anderen BU16800400,Wijk 04 Anderen WK168004,230,125,110,15,30,20,105,50,...,0.4242,1560.0000,2470.0000,0.4242,0.4242,2580.0000,0.4242,3920.0000,0.0,8.0000
9,Verspreide huizen Anderen BU16800409,Wijk 04 Anderen WK168004,30,20,15,5,0,5,10,0,...,0.4242,0.4242,0.4242,0.4242,0.4242,0.4242,0.4242,0.4242,0.0,0.4242
